In [ ]:
#!pip install tensorflow_text

In [ ]:
import requests
import nltk
import spacy
import tensorflow as tf
# import tensorflow_text as tf_text
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd
import numpy as np
from tensorflow.keras import utils

In [ ]:
%tensorflow_version 2.x
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, MaxPooling1D, Conv1D, GlobalMaxPooling1D, Dropout, LSTM, GRU
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/Colab Notebooks/Neural network Article/Pickle'

groups = pd.read_excel('dataset_groups_delivery.xlsx')

Mounted at /content/gdrive
/content/gdrive/MyDrive/Colab Notebooks/Neural network Article/Pickle


In [ ]:
groups = groups.dropna()
groups

,ID_группы,Название,ID_города,Город,Признак,Описание
0,219380671,Каратэ Суши - кафе с доставкой Дзержинск,621,Дзержинск,1,Кафе японской кухни с доставкой по городу Дзер...
1,73662138,Церебро Таргет | продвижение и реклама бизнеса,2,Санкт-Петербург,0,Это официальное сообщество сервиса Церебро Тар...
2,80040324,ЧС Тюмень,147,Тюмень,0,"Предлагайте свои новости в ""предложить новость..."
3,214526981,Амба Пицца,95,Нижний Новгород,1,Доставка вкусной квадратной пиццы по Нижнему Н...
4,213243329,Каратэ Суши - кафе с доставкой Кстово,548,Кстово,1,Кафе японской кухни с доставкой по городу Ксто...
...,...,...,...,...,...,...
288,209005931,Фестиваль ПЛЯЖ,158,Челябинск,0,Что же такое Фестиваль “Пляж” ?\n\n- Это 4 дня...
289,211164663,«Сами Сусами» Тюмень,147,Тюмень,1,Грузинский дом СамиСусами\n\nПонедельник-Четве...
290,211222170,Омар Хайям | ресторан Тюмень,147,Тюмень,0,«Омар Хайям» - сеть ресторанов восточно-европе...
291,211287451,Анна Панкрашова Пародонтолог Мега-Дент Тюмень,147,Тюмень,0,✍️ Моя цель-Здоровье\n🙋🏻‍♀️ Применяю технологи...


# Разделение данных на обучающее и тестовое множества

In [ ]:
#вспомогательная функция деления датафреймов
def split_dataframe(dataframe, test_proportion):
    total_size = len(dataframe)
    test_size = int(total_size * test_proportion)
    indices = np.arange(total_size)
    np.random.shuffle(indices)
    train_indices = indices[0:total_size-test_size]
    test_indices = indices[total_size - test_size:]
    return dataframe.iloc[train_indices], dataframe.iloc[test_indices]

In [ ]:
train, test = split_dataframe(groups, 0.3)

In [ ]:
train, test = train.reset_index(drop=True), test.reset_index(drop=True)

# Выделяем правильные ответы

In [ ]:
nb_classes = 2
y_train = utils.to_categorical(train["Признак"], nb_classes)

In [ ]:
y_train

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.

# Токенизация текста

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
num_words = 100000

In [ ]:
tokenizer = Tokenizer(num_words=num_words)

In [ ]:
names = train['Название']
descrs = train['Описание']
# names = names.reset_index(drop=True)
# descrs = descrs.reset_index(drop=True)
names

0                               SNAILKICK™ | НЕ БЛОГЕР
1                                Караоке-клуб "АРТист"
2                                        RockestraLive
3                          Путешествия "the mounta1ns"
4              Formicant - Термосумки для доставки еды
                            ...                       
194                                         ЗавтраМарс
195    Рекламное производство «Студия 7» Вывеска Пермь
196                                      Паблик Тюмень
197                                           X5 Group
198           ХАН БУЗ | бурятская кухня в Новосибирске
Name: Название, Length: 199, dtype: object

In [ ]:
tokenizer.fit_on_texts(descrs)

In [ ]:
tokenizer.word_index

{'и': 1,
 'в': 2,
 'на': 3,
 'с': 4,
 'по': 5,
 'для': 6,
 'мы': 7,
 'тюмень': 8,
 '00': 9,
 '—': 10,
 'https': 11,
 'не': 12,
 'vk': 13,
 'ru': 14,
 'это': 15,
 'от': 16,
 '7': 17,
 'или': 18,
 'работа': 19,
 'а': 20,
 'вы': 21,
 'до': 22,
 'тюмени': 23,
 'com': 24,
 'нас': 25,
 '1': 26,
 'у': 27,
 'к': 28,
 'о': 29,
 'за': 30,
 'доставка': 31,
 'печать': 32,
 'из': 33,
 '8': 34,
 '•': 35,
 'что': 36,
 '✅': 37,
 'ул': 38,
 '–': 39,
 'только': 40,
 'работы': 41,
 'me': 42,
 '5': 43,
 '23': 44,
 '12': 45,
 'все': 46,
 'также': 47,
 'cc': 48,
 '19': 49,
 'изготовление': 50,
 'пермь': 51,
 '30': 52,
 '10': 53,
 '13': 54,
 'как': 55,
 'вас': 56,
 '3': 57,
 'можно': 58,
 '18': 59,
 '25': 60,
 't': 61,
 '11': 62,
 'россии': 63,
 'г': 64,
 'новости': 65,
 'создание': 66,
 'более': 67,
 'вам': 68,
 'нам': 69,
 'время': 70,
 '24': 71,
 'без': 72,
 'онлайн': 73,
 '👉': 74,
 '17': 75,
 '6': 76,
 '2': 77,
 'рок': 78,
 'которые': 79,
 '72': 80,
 'чтобы': 81,
 'сообщество': 82,
 'окна': 83,
 'то': 84

# Преобразуем описания в числовое представление

In [ ]:
sequences = tokenizer.texts_to_sequences(descrs)

In [ ]:
index = 15

In [ ]:
print(descrs[index])
print(sequences[index])

RAVE PUNK ROCK
[1933, 1934, 940]


In [ ]:
#определение макс длины
max = 0
for i in sequences:
  if len(i) > max: max = len(i)
max

506

In [ ]:
sequences

[[377,
  1619,
  529,
  1,
  298,
  820,
  821,
  11,
  61,
  42,
  529,
  84,
  36,
  378,
  822,
  11,
  1620,
  299,
  529,
  84,
  36,
  378,
  530],
 [1621,
  823,
  6,
  824,
  379,
  39,
  825,
  531,
  379,
  380,
  1622,
  184,
  1623,
  379,
  532,
  533,
  28,
  69,
  2,
  826,
  827,
  15,
  12,
  161,
  379,
  380,
  15,
  85,
  162,
  185,
  1624,
  1625,
  1,
  1626,
  27,
  25,
  2,
  1627,
  21,
  1628,
  1629,
  1630,
  828,
  381,
  1631,
  52,
  1632,
  77,
  1633,
  829,
  231,
  1634,
  830,
  831],
 [382,
  10,
  534,
  1635,
  33,
  1636,
  1637,
  1638,
  1639,
  3,
  383,
  1640,
  1641,
  832,
  78,
  384,
  2,
  833,
  1642,
  1643,
  1644,
  2,
  535,
  385,
  30,
  70,
  1645,
  1646,
  1647,
  1648,
  536,
  5,
  300,
  386,
  834,
  1649,
  3,
  835,
  1650,
  1651,
  1652,
  382,
  2,
  1653,
  70,
  1654,
  1655,
  301,
  1,
  1656,
  1657,
  16,
  384,
  1,
  836,
  387,
  388,
  1658,
  1659,
  1660,
  299,
  1661,
  1662,
  1663,
  1664,
  1665,
  1

# Приводим длины названий к одной длинне

In [ ]:
x_train = pad_sequences(sequences, maxlen = max)
x_train

array([[   0,    0,    0, ...,   36,  378,  530],
       [   0,    0,    0, ..., 1634,  830,  831],
       [   0,    0,    0, ...,  837, 1673, 1674],
       ...,
       [   0,    0,    0, ...,  588, 5689, 5690],
       [   0,    0,    0, ...,  234,   45, 5728],
       [   0,    0,    0, ..., 5755,   14, 5756]], dtype=int32)

# Сверточная нейросеть

In [ ]:
%tensorflow_version 2.x
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, MaxPooling1D, Conv1D, GlobalMaxPooling1D, Dropout, LSTM, GRU
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
model_cnn = Sequential()
model_cnn.add(Embedding(100, 32, input_length=max))
model_cnn.add(Conv1D(350, 8, padding='valid', activation='relu'))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(128, activation='relu'))
model_cnn.add(Dense(2, activation='softmax'))

In [ ]:
model_cnn.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model_cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 506, 32)           3200      
                                                                 
 conv1d (Conv1D)             (None, 499, 350)          89950     
                                                                 
 global_max_pooling1d (Glob  (None, 350)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 128)               44928     
                                                                 
 dense_1 (Dense)             (None, 2)                 258       
                                                                 
Total params: 138336 (540.38 KB)
Trainable params: 138336 (540.38 KB)
Non-trainable params: 0 (0.00 Byte)
________________

Зоздаем Callback (файл для сохранения нейронной сети)

In [ ]:
model_cnn_save_path = 'best_model_cnn.h5'
checkpoint_callback_cnn = ModelCheckpoint(model_cnn_save_path,
                                      monitor='val_accuracy',
                                      save_best_only=True,
                                      verbose=1)

In [ ]:
history_cnn = model_cnn.fit(x_train,
                            y_train,
                            epochs=5,
                            batch_size=128,
                            validation_split=0.1,
                            callbacks=[checkpoint_callback_cnn])

Epoch 1/5


InvalidArgumentError: Graph execution error:

Detected at node sequential/embedding/embedding_lookup defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/l

In [ ]:
plt.plot(history_cnn.history['accuracy'],
         label='Доля верных ответов на обучающем наборе')
plt.plot(history_cnn.history['val_accuracy'],
         label='Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

# Рекурентная сеть LSTM

In [ ]:
model_lstm = Sequential()
model_lstm.add(Embedding(num_words, 32, input_length=max))
model_lstm.add(LSTM(16))
model_lstm.add(Dense(2, activation='softmax'))

In [ ]:
model_lstm.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model_lstm.summary()

In [ ]:
model_lstm_save_path = 'best_model_lstm.h5'
checkpoint_callback_lstm = ModelCheckpoint(model_lstm_save_path,
                                      monitor='val_accuracy',
                                      save_best_only=True,
                                      verbose=1)

In [ ]:
history_lstm = model_lstm.fit(x_train,
                              y_train,
                              epochs=5,
                              batch_size=128,
                              validation_split=0.1,
                              callbacks=[checkpoint_callback_lstm])

In [ ]:
plt.plot(history_lstm.history['accuracy'],
         label='Доля верных ответов на обучающем наборе')
plt.plot(history_lstm.history['val_accuracy'],
         label='Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

# Рекурентная сеть GRU

In [ ]:
model_gru = Sequential()
model_gru.add(Embedding(num_words, 32, input_length=max))
model_gru.add(GRU(16))
model_gru.add(Dense(2, activation='softmax'))

In [ ]:
model_gru.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model_gru.summary()

In [ ]:
model_gru_save_path = 'best_model_gru.h5'
checkpoint_callback_gru = ModelCheckpoint(model_gru_save_path,
                                      monitor='val_accuracy',
                                      save_best_only=True,
                                      verbose=1)

In [ ]:
history_gru = model_gru.fit(x_train,
                              y_train,
                              epochs=5,
                              batch_size=128,
                              validation_split=0.1,
                              callbacks=[checkpoint_callback_gru])

In [ ]:
plt.plot(history_gru.history['accuracy'],
         label='Доля верных ответов на обучающем наборе')
plt.plot(history_gru.history['val_accuracy'],
         label='Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

# Подключаем тестуруемое множество

In [ ]:
test

In [ ]:
test_sequences = tokenizer.texts_to_sequences(test['Описание'])

In [ ]:
x_test = pad_sequences(test_sequences, maxlen=max)

In [ ]:
x_test

In [ ]:
y_test = utils.to_categorical(test['Признак'], nb_classes)

In [ ]:
y_test

# Оцениваем работу сетей на тестовом множестве

# Одномерная сверточная сеть

In [ ]:
model_cnn.load_weights(model_cnn_save_path)

In [ ]:
model_cnn.evaluate(x_test, y_test, verbose=1)

# Рекурентная сеть LSTM

In [ ]:
model_lstm.load_weights(model_lstm_save_path)

In [ ]:
model_lstm.evaluate(x_test, y_test, verbose=1)

# Сеть GRU

In [ ]:
model_gru.load_weights(model_gru_save_path)

In [ ]:
model_gru.evaluate(x_test, y_test, verbose=1)